### Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from dateutil import parser

### Importing the data

In [2]:
raw_train = pd.read_csv('train.csv')
raw_test = pd.read_csv('test_x.csv')

train_df = raw_train.copy()
test_df = raw_test.copy()

/var/folders/q7/w_hh_4ds07n_pl9zc0p1wxbc0000gn/T/ipykernel_8662/1415330426.py:1: DtypeWarning: Columns (13,15,19,30,32,37,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_train = pd.read_csv('train.csv')


### Functions

In [3]:
def analyze_dataframe(df):
    """
    Function to analyze a DataFrame by printing:
    - Shape of the DataFrame
    - Information about each column (non-null count and data type)
    - Descriptive statistics for numerical columns
    - Number of missing values in each column
    - List of all column names

    Parameters:
    df (pd.DataFrame): The DataFrame to analyze
    """
    # Print shape of the DataFrame
    print("SHAPE")
    print(df.shape)
    print("\n----------------------------------------------------\n")
    
    # Print column information (non-null count, data types)
    print(df.info())
    print("\n----------------------------------------------------\n")
    
    # Print descriptive statistics for numerical columns
    print(df.describe())
    print("\n----------------------------------------------------\n")
    
    # Print count of missing values in each column
    print(df.isna().sum())
    print("\n----------------------------------------------------\n")
    
    # Print all column names
    print(df.columns)

In [4]:
def analyze_column(df, column_name):
    """
    Function to analyze a column in a DataFrame by printing:
    - Unique values
    - Value counts (including NaNs)
    - Number of missing values (NaNs)

    Parameters:
    df (pd.DataFrame): The DataFrame containing the column
    column_name (str): The name of the column to analyze
    """
    # Print unique values
    print(f"Unique values in '{column_name}':")
    print(df[column_name].unique())
    print(' ')
    
    # Print value counts (including NaNs)
    print(f"Value counts in '{column_name}' (including NaNs):")
    print(df[column_name].value_counts(dropna=False))
    print(' ')
    
    # Print the number of missing values (NaNs)
    print(f"Number of missing values (NaNs) in '{column_name}':")
    print(df[column_name].isna().sum())
    
    print(' ')
    print(df[column_name].describe())

In [5]:
def standardize_dates(df, date_column):
    """
    Standardizes date formats in the specified column of the DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the date column.
    date_column (str): The name of the column containing dates in various formats.

    Returns:
    pd.DataFrame: DataFrame with the standardized date column.
    """
    def parse_date(date_str):
        try:
            return parser.parse(date_str, dayfirst=True)
        except (ValueError, TypeError):
            return pd.NaT

    # Apply the parsing function
    df['Parsed Date'] = df[date_column].apply(parse_date)

    # Ensure 'Parsed Date' column is of datetime type
    df['Parsed Date'] = pd.to_datetime(df['Parsed Date'], errors='coerce')

    # Convert to standard format
    df['Standardized Date'] = df['Parsed Date'].dt.strftime('%Y-%m-%d')

    # Replace original column and clean up
    df[date_column] = df['Standardized Date']
    df = df.drop(columns=['Parsed Date', 'Standardized Date'])
    
    return df

In [6]:
def plot_boxplot_with_outliers(df, column_name):
    """
    Plots a boxplot of the specified column, highlighting the outliers.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    column_name (str): The name of the column to plot.
    """
    # Create a boxplot
    plt.figure(figsize=(10, 12))
    sns.boxplot(data=df, y=column_name)
    
    # Add title and labels
    plt.title(f'Boxplot of {column_name}')
    plt.ylabel(column_name)
    
    # Show the plot
    plt.show()

In [7]:
def list_outlier_values(df, column_name):
    """
    Lists outlier values in a specified column of a DataFrame using the IQR method.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    column_name (str): The name of the column to check for outliers.

    Returns:
    list: A list of outlier values in the specified column.
    """
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    
    # Calculate the Interquartile Range (IQR)
    IQR = Q3 - Q1
    
    # Determine the outlier boundaries
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Detect outlier values
    outlier_values = df[(df[column_name] < lower_bound) | (df[column_name] > upper_bound)][column_name].tolist()
    
    return outlier_values

In [8]:
cities = [
    "Adana", "Adıyaman", "Afyonkarahisar", "Ağrı", "Aksaray", "Amasya", "Ankara", "Antalya",
    "Ardahan", "Artvin", "Aydın", "Balıkesir", "Bartın", "Batman", "Bayburt", "Bilecik", "Bingöl",
    "Bitlis", "Bolu", "Burdur", "Bursa", "Çanakkale", "Çankırı", "Çorum", "Denizli", "Diyarbakır",
    "Düzce", "Edirne", "Elazığ", "Erzincan", "Erzurum", "Eskişehir", "Gaziantep", "Giresun", "Gümüşhane",
    "Hakkari", "Hatay", "Iğdır", "Isparta", "İstanbul", "İzmir", "Kahramanmaraş", "Karabük", "Karaman",
    "Kars", "Kastamonu", "Kayseri", "Kilis", "Kırıkkale", "Kırklareli", "Kırşehir", "Kocaeli", "Konya",
    "Kütahya", "Malatya", "Manisa", "Mardin", "Mersin", "Muğla", "Muş", "Nevşehir", "Niğde", "Ordu",
    "Osmaniye", "Rize", "Sakarya", "Samsun", "Siirt", "Sinop", "Sivas", "Şanlıurfa", "Şırnak", "Tekirdağ",
    "Tokat", "Trabzon", "Tunceli", "Uşak", "Van", "Yalova", "Yozgat", "Zonguldak"
]

def fill_city(city):
    if pd.isna(city):
        return None
    
    if isinstance(city, str):
        city = city.lower()
        city = city.strip()
        
        for n in cities:
            if n.lower() in city:
                return n
    return None

In [9]:
def standardize_value(value):
    if isinstance(value, str):
        value = value.replace(' ', '')  # Remove spaces
        value = value.replace('-', '-')  # Ensure hyphen consistency
        value = value.replace('vealtı', 've altı')  # Handle specific terms
        return value
    return value

In [10]:
def extract_numbers(text):
    if isinstance(text, str):
        numbers = re.findall(r'\d+', text)
        return int(max(numbers, key=int)) if numbers else 0
    return 0

### 1-) Exploratory Data Analysis

In [11]:
train_df.head()

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2014,52.0,Erkek,4/6/1994,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,...,Diğer,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,0
1,2014,30.0,Erkek,6/11/1993,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,...,0,Hayır,NaN,Evet,Evet,Bilkent Cyberparkta bir şirkette Türkiye nin i...,NaN,NaN,NaN,1
2,2014,18.0,Erkek,1/15/1986,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,2
3,2014,40.0,Erkek,6/4/1991,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,3
4,2014,24.0,Erkek,2 Kasim 1992,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,4


In [12]:
analyze_dataframe(train_df)

SHAPE
(65125, 44)

----------------------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65125 entries, 0 to 65124
Data columns (total 44 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Basvuru Yili                                               65125 non-null  int64  
 1   Degerlendirme Puani                                        65124 non-null  float64
 2   Cinsiyet                                                   64956 non-null  object 
 3   Dogum Tarihi                                               64948 non-null  object 
 4   Dogum Yeri                                                 64334 non-null  object 
 5   Ikametgah Sehri                                            63088 non-null  object 
 6   Universite Adi                                             64993 non-null  object 
 7   Universite Turu      

## 2-) Preproccessing 

### 2.1-) Degerlendirme Puani

In [13]:
analyze_column(train_df, 'Degerlendirme Puani')

Unique values in 'Degerlendirme Puani':
[ 52.  30.  18.  40.  24.  37.  31.  28.  26.  58.  34.  33.  48.  10.
   8.  54.  14.  44.  60.  42.  22.  16.  29.  46.  41.  50.  53.  36.
  32.  38.  20.  12.  51.  65.  56.  61.  62.  49.  70.  43.  23.  64.
  59.  57.  63.  71.  98.  21.  66.  27.  39.  68.  85.  45.  67.  47.
  78.  19.  13.  90.  72.  77.  55.   6.  79.  76.  75.  74.  73.  17.
  80.  84.  86.  82.  88.  92.  94.  96. 102.   7.   4.  15.  35.  25.
   5.  11.   9.   3.   2.   1.   0.  69.  81.  89.  83. 100.  95.  nan]
 
Value counts in 'Degerlendirme Puani' (including NaNs):
Degerlendirme Puani
28.0     2063
22.0     2041
18.0     1995
24.0     1785
12.0     1740
         ... 
83.0        2
100.0       2
89.0        1
95.0        1
NaN         1
Name: count, Length: 98, dtype: int64
 
Number of missing values (NaNs) in 'Degerlendirme Puani':
1
 
count    65124.000000
mean        32.086466
std         18.139239
min          0.000000
25%         18.000000
50%         29.000

In [14]:
train_df['Degerlendirme Puani'] = train_df['Degerlendirme Puani'].fillna(train_df['Degerlendirme Puani'].median())

In [15]:
analyze_column(train_df, 'Degerlendirme Puani')

Unique values in 'Degerlendirme Puani':
[ 52.  30.  18.  40.  24.  37.  31.  28.  26.  58.  34.  33.  48.  10.
   8.  54.  14.  44.  60.  42.  22.  16.  29.  46.  41.  50.  53.  36.
  32.  38.  20.  12.  51.  65.  56.  61.  62.  49.  70.  43.  23.  64.
  59.  57.  63.  71.  98.  21.  66.  27.  39.  68.  85.  45.  67.  47.
  78.  19.  13.  90.  72.  77.  55.   6.  79.  76.  75.  74.  73.  17.
  80.  84.  86.  82.  88.  92.  94.  96. 102.   7.   4.  15.  35.  25.
   5.  11.   9.   3.   2.   1.   0.  69.  81.  89.  83. 100.  95.]
 
Value counts in 'Degerlendirme Puani' (including NaNs):
Degerlendirme Puani
28.0     2063
22.0     2041
18.0     1995
24.0     1785
12.0     1740
         ... 
85.0        2
83.0        2
100.0       2
89.0        1
95.0        1
Name: count, Length: 97, dtype: int64
 
Number of missing values (NaNs) in 'Degerlendirme Puani':
0
 
count    65125.000000
mean        32.086418
std         18.139104
min          0.000000
25%         18.000000
50%         29.000000
7

### 2.2-) Cinsiyet

In [16]:
analyze_column(train_df, 'Cinsiyet')

Unique values in 'Cinsiyet':
['Erkek' 'Kadın' 'ERKEK' 'Belirtmek istemiyorum' nan]
 
Value counts in 'Cinsiyet' (including NaNs):
Cinsiyet
Kadın                    32077
Erkek                    22752
ERKEK                    10012
NaN                        169
Belirtmek istemiyorum      115
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Cinsiyet':
169
 
count     64956
unique        4
top       Kadın
freq      32077
Name: Cinsiyet, dtype: object


In [17]:
train_df['Cinsiyet'] = train_df['Cinsiyet'].replace('ERKEK', 'Erkek')
train_df['Cinsiyet'].isin(['ERKEK']).any()

False

In [18]:
# Replacing the NaN values with 'Erkek'(Mod)

train_df['Cinsiyet'] = train_df['Cinsiyet'].fillna('Erkek')
train_df['Cinsiyet'].value_counts(dropna = False)

Cinsiyet
Erkek                    32933
Kadın                    32077
Belirtmek istemiyorum      115
Name: count, dtype: int64

In [19]:
analyze_column(train_df, 'Cinsiyet')

Unique values in 'Cinsiyet':
['Erkek' 'Kadın' 'Belirtmek istemiyorum']
 
Value counts in 'Cinsiyet' (including NaNs):
Cinsiyet
Erkek                    32933
Kadın                    32077
Belirtmek istemiyorum      115
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Cinsiyet':
0
 
count     65125
unique        3
top       Erkek
freq      32933
Name: Cinsiyet, dtype: object


In [20]:
label_encoder = LabelEncoder()
train_df['Cinsiyet'] = label_encoder.fit_transform(train_df['Cinsiyet'])

In [21]:
analyze_column(train_df, 'Cinsiyet')

Unique values in 'Cinsiyet':
[1 2 0]
 
Value counts in 'Cinsiyet' (including NaNs):
Cinsiyet
1    32933
2    32077
0      115
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Cinsiyet':
0
 
count    65125.000000
mean         1.490779
std          0.503439
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          2.000000
Name: Cinsiyet, dtype: float64


### 2.3-) Dogum Tarihi

In [22]:
standardize_dates(train_df, 'Dogum Tarihi')

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2014,52.0,1,1994-06-04,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,...,Diğer,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,0
1,2014,30.0,1,1993-11-06,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,...,0,Hayır,NaN,Evet,Evet,Bilkent Cyberparkta bir şirkette Türkiye nin i...,NaN,NaN,NaN,1
2,2014,18.0,1,1986-01-15,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,2
3,2014,40.0,1,1991-04-06,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,3
4,2014,24.0,1,NaN,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,1,2003-06-18,İstanbul,İstanbul,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65120
65121,2022,0.0,1,2002-04-18,Hatay,Muğla,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65121
65122,2022,0.0,1,1999-10-23,Mersin,Mersin,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65122
65123,2022,0.0,1,2000-11-17,Samsun,Samsun,Karabük Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65123


In [23]:
analyze_dataframe(train_df)

SHAPE
(65125, 46)

----------------------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65125 entries, 0 to 65124
Data columns (total 46 columns):
 #   Column                                                     Non-Null Count  Dtype         
---  ------                                                     --------------  -----         
 0   Basvuru Yili                                               65125 non-null  int64         
 1   Degerlendirme Puani                                        65125 non-null  float64       
 2   Cinsiyet                                                   65125 non-null  int64         
 3   Dogum Tarihi                                               62009 non-null  object        
 4   Dogum Yeri                                                 64334 non-null  object        
 5   Ikametgah Sehri                                            63088 non-null  object        
 6   Universite Adi                                       

In [24]:
train_df['Dogum Tarihi'] = train_df['Parsed Date']

In [25]:
train_df.drop(columns=['Parsed Date', 'Standardized Date'], inplace=True)

In [26]:
reference_date = pd.Timestamp('2023-12-31')

# Calculate the age by dividing the timedelta by one year (in days)
train_df['Dogum Tarihi'] = (reference_date - train_df['Dogum Tarihi']).dt.days / 365.25

print(train_df[['Dogum Tarihi']])

       Dogum Tarihi
0         29.574264
1         30.149213
2         37.957563
3         32.736482
4               NaN
...             ...
65120     20.536619
65121     21.702943
65122     24.188912
65123     23.118412
65124     23.676934

[65125 rows x 1 columns]


In [27]:
analyze_column(train_df, 'Dogum Tarihi')

Unique values in 'Dogum Tarihi':
[29.5742642  30.14921287 37.95756331 ... 20.14784394 20.72553046
 20.13415469]
 
Value counts in 'Dogum Tarihi' (including NaNs):
Dogum Tarihi
 NaN          3116
-46.006845    1655
 23.997262     290
 22.995209     247
 24.996578     230
              ... 
 32.632444       1
 31.008898       1
 34.017796       1
 7.986311        1
 20.134155       1
Name: count, Length: 5050, dtype: int64
 
Number of missing values (NaNs) in 'Dogum Tarihi':
3116
 
count    62009.000000
mean        24.361789
std         12.127657
min        -46.176591
25%         23.635866
50%         25.924709
75%         28.498289
max        135.351129
Name: Dogum Tarihi, dtype: float64


In [28]:
outlier_values = list_outlier_values(train_df, 'Dogum Tarihi')
print(outlier_values)

[37.95756331279945, 36.15605749486653, 53.27857631759069, 53.9958932238193, 37.946611909650926, 37.96303901437371, 37.516769336071185, 53.9958932238193, 53.9958932238193, 7.5482546201232035, 7.397672826830938, 7.17864476386037, 7.321013004791239, 6.685831622176591, 2.0424366872005475, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.00684462696783, -46.0068446269

In [29]:
train_df.loc[train_df['Dogum Tarihi'] < 15, 'Dogum Tarihi'] = 0
train_df.loc[train_df['Dogum Tarihi'] > 34, 'Dogum Tarihi'] = 0

In [30]:
outlier_values = list_outlier_values(train_df, 'Dogum Tarihi')
print(outlier_values)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [31]:
#Calculate the mean value of the column, ignoring NaNs
mean_value = train_df['Dogum Tarihi'].replace(0, np.nan).mean()

#Replace NaNs and 0s with the mean value
train_df['Dogum Tarihi'] = train_df['Dogum Tarihi'].replace(0, np.nan).fillna(mean_value)

In [32]:
analyze_column(train_df, 'Dogum Tarihi')

Unique values in 'Dogum Tarihi':
[29.5742642  30.14921287 26.25168006 ... 20.14784394 20.72553046
 20.13415469]
 
Value counts in 'Dogum Tarihi' (including NaNs):
Dogum Tarihi
26.251680    5058
23.997262     290
22.995209     247
24.996578     230
25.995893     228
             ... 
31.586585       1
30.587269       1
30.603696       1
31.600274       1
20.134155       1
Name: count, Length: 4827, dtype: int64
 
Number of missing values (NaNs) in 'Dogum Tarihi':
0
 
count    65125.000000
mean        26.251680
std          2.890494
min         17.111567
25%         23.975359
50%         26.251680
75%         28.347707
max         33.995893
Name: Dogum Tarihi, dtype: float64


### 2.4-) Dogum Yeri

In [33]:
train_df['Dogum Yeri'] = train_df['Dogum Yeri'].apply(fill_city)

In [34]:
analyze_column(train_df, 'Dogum Yeri')

Unique values in 'Dogum Yeri':
['Ankara' None 'Samsun' 'Diyarbakır' 'İstanbul' 'Erzincan' 'Sivas'
 'Kayseri' 'Uşak' 'Rize' 'Erzurum' 'Konya' 'İzmir' 'Trabzon' 'Kars'
 'Kırşehir' 'Adana' 'Sakarya' 'Muğla' 'Hatay' 'Mersin' 'Malatya' 'Manisa'
 'Çorum' 'Çanakkale' 'Antalya' 'Çankırı' 'Eskişehir' 'Bartın' 'Karaman'
 'Van' 'Bingöl' 'Giresun' 'Ordu' 'Bursa' 'Kütahya' 'Karabük' 'Bilecik'
 'Siirt' 'Adıyaman' 'Kırklareli' 'Aydın' 'Elazığ' 'Zonguldak' 'Kocaeli'
 'Balıkesir' 'Tekirdağ' 'Tokat' 'Şanlıurfa' 'Kastamonu' 'Edirne' 'Hakkari'
 'Ağrı' 'Kahramanmaraş' 'Nevşehir' 'Kırıkkale' 'Isparta' 'Kilis' 'Batman'
 'Osmaniye' 'Gaziantep' 'Denizli' 'Mardin' 'Bolu' 'Niğde' 'Amasya'
 'Aksaray' 'Düzce' 'Yozgat' 'Sinop' 'Yalova' 'Bayburt' 'Muş' 'Bitlis'
 'Tunceli' 'Gümüşhane' 'Burdur' 'Şırnak' 'Iğdır' 'Ardahan' 'Artvin'
 'Afyonkarahisar']
 
Value counts in 'Dogum Yeri' (including NaNs):
Dogum Yeri
İstanbul     8999
None         8227
Ankara       4275
İzmir        2219
Bursa        1931
             ... 
Bile

In [35]:
# Replace 'None' with 'İstanbul'
train_df['Dogum Yeri'] = train_df['Dogum Yeri'].fillna('İstanbul')

In [36]:
analyze_column(train_df, 'Dogum Yeri')

Unique values in 'Dogum Yeri':
['Ankara' 'İstanbul' 'Samsun' 'Diyarbakır' 'Erzincan' 'Sivas' 'Kayseri'
 'Uşak' 'Rize' 'Erzurum' 'Konya' 'İzmir' 'Trabzon' 'Kars' 'Kırşehir'
 'Adana' 'Sakarya' 'Muğla' 'Hatay' 'Mersin' 'Malatya' 'Manisa' 'Çorum'
 'Çanakkale' 'Antalya' 'Çankırı' 'Eskişehir' 'Bartın' 'Karaman' 'Van'
 'Bingöl' 'Giresun' 'Ordu' 'Bursa' 'Kütahya' 'Karabük' 'Bilecik' 'Siirt'
 'Adıyaman' 'Kırklareli' 'Aydın' 'Elazığ' 'Zonguldak' 'Kocaeli'
 'Balıkesir' 'Tekirdağ' 'Tokat' 'Şanlıurfa' 'Kastamonu' 'Edirne' 'Hakkari'
 'Ağrı' 'Kahramanmaraş' 'Nevşehir' 'Kırıkkale' 'Isparta' 'Kilis' 'Batman'
 'Osmaniye' 'Gaziantep' 'Denizli' 'Mardin' 'Bolu' 'Niğde' 'Amasya'
 'Aksaray' 'Düzce' 'Yozgat' 'Sinop' 'Yalova' 'Bayburt' 'Muş' 'Bitlis'
 'Tunceli' 'Gümüşhane' 'Burdur' 'Şırnak' 'Iğdır' 'Ardahan' 'Artvin'
 'Afyonkarahisar']
 
Value counts in 'Dogum Yeri' (including NaNs):
Dogum Yeri
İstanbul     17226
Ankara        4275
İzmir         2219
Bursa         1931
Adana         1914
             ...  
Bil

In [37]:
train_df['Dogum Yeri'] = label_encoder.fit_transform(train_df['Dogum Yeri'])

In [38]:
analyze_column(train_df, 'Dogum Yeri')

Unique values in 'Dogum Yeri':
[ 5 77 61 22 26 64 41 69 59 27 44 78 67 39 48  0 60 53 33 52 49 50 76 74
  6 75 28 12 38 70 16 30 57 20 45 37 15 62  1 46  9 25 73 43 11 65 66 79
 40 24 32 10 36 55 47 34 42 13 58 29 21 51 18 56  4  3 23 72 63 71 14 54
 17 68 31 19 80 35  7  8  2]
 
Value counts in 'Dogum Yeri' (including NaNs):
Dogum Yeri
77    17226
5      4275
78     2219
20     1931
0      1914
      ...  
15      100
75       97
31       82
14       79
68       55
Name: count, Length: 81, dtype: int64
 
Number of missing values (NaNs) in 'Dogum Yeri':
0
 
count    65125.000000
mean        47.693390
std         27.556491
min          0.000000
25%         22.000000
50%         51.000000
75%         77.000000
max         80.000000
Name: Dogum Yeri, dtype: float64


### 2.5-) Ikametgah Sehri

In [39]:
analyze_column(train_df, 'Ikametgah Sehri')

Unique values in 'Ikametgah Sehri':
['Ankara' 'İstanbul' 'Esenler/İstanbul' 'Kayseri' 'Bahçelievler/Ankara'
 'İSTANBUL' 'İzmir' 'Kahramanmaraş' 'Üsküdar/İstanbul' 'İZMİR' 'Trabzon'
 'Kars' 'Kırşehir' 'Gebze/Kocaeli' 'Buca/İzmir' 'Sarıyer/İstanbul' 'Ordu'
 'Osmaniye' nan 'Maltepe/İstanbul' 'Antalya' 'Florya/İstanbul'
 'Gaziosmanpaşa/İstanbul' 'Istanbul' 'Etimesgut/Ankara' 'Konya' 'ANKARA'
 'Bartın' 'Girne' 'Bingöl' 'Mersin' 'KAYSERİ' 'BURSA' 'Manisa' 'Samsun'
 'GÖLCÜK' 'Diyarbakır' 'istanbul' 'Kırklareli' 'KOCAELİ' 'AYDIN' 'Muğla'
 'Zonguldak' 'Erzurum' 'GAZİMAĞUSA' 'Bursa' 'DENİZLİ' 'Batman' 'Gaziantep'
 'Aydın' 'Kocaeli' 'Gazimağusa' 'Adana' 'Balıkesir' 'Şanlıurfa' 'Sakarya'
 'ANTALYA' 'Izmir' 'ADANA' 'NEVŞEHİR' 'KONYA' 'ISTANBUL' 'Niğde' 'Arhavi'
 'ankara' 'AKSARAY' 'Güzelyurt' 'Kütahya' 'AFYONKARAHİSAR' 'Uşak'
 'SAKARYA' 'Bolu' 'mersin' 'izmir' 'Aksaray' 'Kastamonu' 'K.maraş'
 'Tekirdağ' 'Düzce' '------' 'Sinop' 'Elazığ' 'Kırıkkale' 'Isparta' 'Ağrı'
 'Giresun' 'Hatay' 'Denizli' 'Çan

In [40]:
train_df['Ikametgah Sehri'] = train_df['Ikametgah Sehri'].apply(fill_city)
train_df['Ikametgah Sehri'] = train_df['Ikametgah Sehri'].fillna('İstanbul')
train_df['Ikametgah Sehri'] = label_encoder.fit_transform(train_df['Ikametgah Sehri'])
analyze_column(train_df, 'Ikametgah Sehri')

Unique values in 'Ikametgah Sehri':
[ 5 77 41 78 36 67 39 48 43 57 58  6 44 12 16 52 20 50 61 22 46 53 73 27
 13 29  9  0 11 79 60 56  3 45 69 18 40 65 23 63 25 47 34 10 30 33 21 74
 62 64 49 80 32 51 70 26 14 72 76 37 17  4 68 24  7 66 71 38 19  8 28 59
 35 54  1 15  2 55 31 75 42]
 
Value counts in 'Ikametgah Sehri' (including NaNs):
Ikametgah Sehri
77    24802
5      6602
78     3104
20     1794
43     1472
      ...  
68       65
42       59
8        55
75       49
7        33
Name: count, Length: 81, dtype: int64
 
Number of missing values (NaNs) in 'Ikametgah Sehri':
0
 
count    65125.000000
mean        50.942326
std         28.693058
min          0.000000
25%         23.000000
50%         62.000000
75%         77.000000
max         80.000000
Name: Ikametgah Sehri, dtype: float64


### 2.6-) Universite Adi

In [41]:
analyze_column(train_df, 'Universite Adi')
train_df['Universite Adi'] = train_df['Universite Adi'].fillna('i̇stanbul üni̇versi̇tesi̇')
train_df['Universite Adi'] = label_encoder.fit_transform(train_df['Universite Adi'])
analyze_column(train_df, 'Universite Adi')

Unique values in 'Universite Adi':
['İHSAN DOĞRAMACI BİLKENT' 'ULUSLARARASI KIBRIS ÜNİVERSİTESİ'
 'İSTANBUL ŞEHİR ÜNİVERSİTESİ' 'TURGUT ÖZAL ÜNİVERSİTESİ'
 'İSTANBUL TİCARET ÜNİVERSİTESİ' 'İSTANBUL MEDİPOL ÜNİVERSİTESİ'
 'FATİH SULTAN MEHMET VAKIF' 'BAHÇEŞEHİR ÜNİVERSİTESİ'
 'NUH NACİ YAZGAN ÜNİVERSİTESİ' 'FATİH ÜNİVERSİTESİ'
 'BAŞKENT ÜNİVERSİTESİ' 'ÇAĞ ÜNİVERSİTESİ' 'İZMİR ÜNİVERSİTESİ'
 'ZİRVE ÜNİVERSİTESİ' 'YAŞAR ÜNİVERSİTESİ' 'İSTANBUL AREL ÜNİVERSİTESİ'
 'TED ÜNİVERSİTESİ' 'YENİ YÜZYIL ÜNİVERSİTESİ'
 'İZMİR EKONOMİ ÜNİVERSİTESİ' 'İSTANBUL BİLGİ ÜNİVERSİTESİ'
 'İSTANBUL GELİŞİM ÜNİVERSİTESİ' 'BEYKENT ÜNİVERSİTESİ'
 'AVRASYA ÜNİVERSİTESİ' 'OKAN ÜNİVERSİTESİ' 'KOÇ ÜNİVERSİTESİ'
 'HALİÇ ÜNİVERSİTESİ' 'IŞIK ÜNİVERSİTESİ' 'ATILIM ÜNİVERSİTESİ'
 'NİŞANTAŞI ÜNİVERSİTESİ' 'İSTANBUL AYDIN ÜNİVERSİTESİ'
 'UFUK ÜNİVERSİTESİ' 'SABANCI ÜNİVERSİTESİ'
 'İSTANBUL 29 MAYIS ÜNİVERSİTESİ' 'MALTEPE ÜNİVERSİTESİ'
 'ULUSLARARASI ANTALYA ÜNİVERSİTESİ' 'TOBB EKONOMİ VE TEKNOLOJİ'
 'KTO KARATAY ÜNİVERSİTE

### 2.7-) Universite Turu

In [42]:
universite_mapping = {
    'DEVLET': 'Devlet',
    'ÖZEL': 'Özel'
}

train_df['Universite Turu'] = train_df['Universite Turu'].replace(universite_mapping)
train_df['Universite Turu'] = train_df['Universite Turu'].fillna('Devlet')
train_df['Universite Turu'] = label_encoder.fit_transform(train_df['Universite Turu'])
analyze_column(train_df, 'Universite Turu')

Unique values in 'Universite Turu':
[1 0]
 
Value counts in 'Universite Turu' (including NaNs):
Universite Turu
0    53517
1    11608
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Universite Turu':
0
 
count    65125.000000
mean         0.178242
std          0.382719
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: Universite Turu, dtype: float64


### 2.7-) Burs Yuzdesi

In [43]:
train_df = train_df.rename(columns={'Burslu ise Burs Yuzdesi': 'Burs Yuzdesi'})
train_df['Burs Yuzdesi'] = train_df['Burs Yuzdesi'].fillna(0)
analyze_column(train_df, 'Burs Yuzdesi')

Unique values in 'Burs Yuzdesi':
[100.   99.   98.   91.5  90.   88.   87.5  85.   82.5  80.   75.   74.
  72.   70.   69.   67.5  67.   65.   64.   63.   62.5  62.   60.   57.5
  57.   55.   50.   49.   48.   46.   45.   41.5  41.   40.   37.   35.
  32.   30.   25.   20.   11.   10.    8.    7.    6.    5.    4.    3.
   2.    1.    0. ]
 
Value counts in 'Burs Yuzdesi' (including NaNs):
Burs Yuzdesi
0.0      54857
100.0     5049
50.0      3149
75.0      1002
25.0       785
10.0        50
60.0        27
1.0         26
65.0        17
80.0        15
5.0         14
30.0        14
85.0        11
2.0         10
55.0         8
62.5         7
62.0         7
35.0         7
45.0         6
7.0          6
70.0         5
99.0         5
20.0         5
40.0         4
90.0         4
63.0         4
6.0          4
3.0          2
8.0          2
98.0         2
46.0         1
82.5         1
69.0         1
88.0         1
64.0         1
4.0          1
87.5         1
57.5         1
57.0         1
91.5     

### 2.8-) Burs Aliyor mu?

In [44]:
burs_mapping = {
    'hayır': 'Hayır',
    'evet': 'Evet',
    'EVET': 'Evet'
}

train_df['Burs Aliyor mu?'] = train_df['Burs Aliyor mu?'].replace(burs_mapping)
train_df['Burs Aliyor mu?'] = label_encoder.fit_transform(train_df['Burs Aliyor mu?'])
analyze_column(train_df, 'Burs Aliyor mu?')

Unique values in 'Burs Aliyor mu?':
[0 1]
 
Value counts in 'Burs Aliyor mu?' (including NaNs):
Burs Aliyor mu?
1    52250
0    12875
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Burs Aliyor mu?':
0
 
count    65125.000000
mean         0.802303
std          0.398265
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: Burs Aliyor mu?, dtype: float64


### 2.9-) Bölüm

In [45]:
department_mapping = {
    'Elektrik Elektronik Mühendisliği': 'Elektrik-Elektronik Mühendisliği',
    'Bilgisayar Mühendisliği': 'Bilgisayar Mühendisliği',
    'Endüstri Mühendisliği': 'Endüstri Mühendisliği',
    'İşletme': 'İşletme',
    'Ekonomi': 'Ekonomi',
    'İktisat': 'İktisat',
    'Matematik': 'Matematik',
    'Siyaset Bilimi ve Kamu Yönetimi': 'Siyaset Bilimi ve Kamu Yönetimi',
    'Siyaset Bilimi ve Uluslararası İlişkiler': 'Siyaset Bilimi ve Uluslararası İlişkiler',
    'Hukuk': 'Hukuk',
    'Psikoloji': 'Psikoloji',
    'Biyomedikal Mühendisliği': 'Biyomedikal Mühendisliği',
    'Yönetim Bilişim Sistemleri': 'Yönetim Bilişim Sistemleri',
    'Maliye': 'Maliye',
    'Yazılım Mühendisliği': 'Yazılım Mühendisliği',
    'Uluslararası Ticaret': 'Uluslararası Ticaret',
    'Felsefe': 'Felsefe',
    'Tarih': 'Tarih',
    'Makine Mühendisliği': 'Makine Mühendisliği',
    'Mekatronik Mühendisliği': 'Mekatronik Mühendisliği',
    'Uluslararası İlişkiler': 'Uluslararası İlişkiler',
    'Tıp Fakültesi': 'Tıp Fakültesi',
    'Moleküler Biyoloji ve Genetik': 'Moleküler Biyoloji ve Genetik',
    'Eczacılık': 'Eczacılık',
    'İnşaat Mühendisliği': 'İnşaat Mühendisliği',
    'İşletme Mühendisliği': 'İşletme Mühendisliği',
    'Gazetecilik': 'Gazetecilik',
    'Hukuk Fakültesi': 'Hukuk Fakültesi',
    'Çevre Mühendisliği': 'Çevre Mühendisliği',
    'Beslenme ve Diyetetik': 'Beslenme ve Diyetetik',
    'Hemşirelik': 'Hemşirelik',
    'Elektronik ve Haberleşme Mühendisliği': 'Elektronik ve Haberleşme Mühendisliği',
    'Rehberlik ve Psikolojik Danışmanlık': 'Rehberlik ve Psikolojik Danışmanlık',
    'Türk Dili ve Edebiyatı': 'Türk Dili ve Edebiyatı',
    'Fizyoterapi ve Rehabilitasyon': 'Fizyoterapi ve Rehabilitasyon',
    'İngiliz Dili ve Edebiyatı': 'İngiliz Dili ve Edebiyatı',
    'Halkla İlişkiler ve Tanıtım': 'Halkla İlişkiler ve Tanıtım',
    'Sosyal Hizmet': 'Sosyal Hizmet',
    'Gıda Mühendisliği': 'Gıda Mühendisliği',
    'Kimya Mühendisliği': 'Kimya Mühendisliği',
    'İstatistik': 'İstatistik',
    'Kimya': 'Kimya',
    'Sosyoloji': 'Sosyoloji',
    'İngilizce Öğretmenliği': 'İngilizce Öğretmenliği',
    'Türkçe Öğretmenliği': 'Türkçe Öğretmenliği',
    'Sınıf Öğretmenliği': 'Sınıf Öğretmenliği',
    'Kamu Yönetimi': 'Kamu Yönetimi',
    'Okul Öncesi Öğretmenliği': 'Okul Öncesi Öğretmenliği',
    'Bilgisayar Mühendisliği(İngilizce)': 'Bilgisayar Mühendisliği',
    'İşletme(İngilizce)': 'İşletme',
    'Diş Hekimliği': 'Diş Hekimliği',
    'Fen Bilgisi Öğretmenliği': 'Fen Bilgisi Öğretmenliği',
    'Metalurji ve Malzeme Mühendisliği': 'Metalurji ve Malzeme Mühendisliği',
    'Endüstri Mühendisliği(İngilizce)': 'Endüstri Mühendisliği',
    'Elektrik-Elektronik Mühendisliği(İngilizce)': 'Elektrik-Elektronik Mühendisliği',
    'İlahiyat': 'İlahiyat',
    'Coğrafya': 'Coğrafya',
    'Makine Mühendisliği(İngilizce)': 'Makine Mühendisliği',
    'Ekonometri': 'Ekonometri',
    'İlköğretim Matematik Öğretmenliği': 'İlköğretim Matematik Öğretmenliği',
    'Elektrik Mühendisliği': 'Elektrik Mühendisliği',
    'Bilgisayar ve Öğretim Teknolojileri Öğretmenliği': 'Bilgisayar ve Öğretim Teknolojileri Öğretmenliği',
    'Çalışma Ekonomisi ve Endüstri İlişkileri': 'Çalışma Ekonomisi ve Endüstri İlişkileri'
}

# Map department names to standardized names
train_df['Bölüm'] = train_df['Bölüm'].map(department_mapping)


In [46]:
most_common_dep = train_df['Bölüm'].value_counts().head(75).index

train_df['Bölüm'] = train_df['Bölüm'].apply(lambda x: x if x in most_common_dep else "Other")
train_df['Bölüm'] = label_encoder.fit_transform(train_df['Bölüm'])

In [47]:
analyze_column(train_df, 'Bölüm')

Unique values in 'Bölüm':
[32 10  1 20 35 12 33  3 47 26 57 46  8 27 44 13 36 50 40 25 28 45 43 30
  6 55 58 16 21 49  0 19 11 34 41 15 53 18 37 17 24 56 23 38 54 42 39 22
 31  5 14 29 51  4  7 52  9  2 48]
 
Value counts in 'Bölüm' (including NaNs):
Bölüm
32    36962
12     2660
57     2598
1      2140
20     1769
25     1428
50     1064
55      897
10      751
45      743
33      687
19      670
35      539
6       522
51      521
38      507
26      472
5       421
34      407
47      396
36      378
27      377
24      375
40      316
28      315
17      310
41      298
46      287
11      279
29      274
48      267
39      266
30      264
22      258
53      257
54      250
8       247
7       242
18      237
23      236
49      234
31      228
58      204
52      200
43      190
37      186
3       170
42      166
56      165
16      164
44      162
0       156
9       153
2       148
14      145
4       143
21      142
15      142
13      140
Name: count, dtype: int64
 
Number 

### 2.10-) Universite Kacinci Sinif

In [48]:
sinif_mapping = {
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    'Tez': 7,
    'Yüksek Lisans': 7,
    'Mezun': 0,
    'hazırlık': 0,
    'Hazırlık': 0,
    '0': 0
}

train_df = train_df.rename(columns={'Universite Kacinci Sinif': 'Sinif'})
train_df['Sinif'] = train_df['Sinif'].replace(sinif_mapping)
train_df['Sinif'] = train_df['Sinif'].fillna(2)
# Explicitly convert the column to integer type
train_df['Sinif'] = train_df['Sinif'].astype(int)

/var/folders/q7/w_hh_4ds07n_pl9zc0p1wxbc0000gn/T/ipykernel_8662/1781766906.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['Sinif'] = train_df['Sinif'].replace(sinif_mapping)


In [49]:
analyze_column(train_df, 'Sinif')

Unique values in 'Sinif':
[3 1 2 4 0 7 5 6]
 
Value counts in 'Sinif' (including NaNs):
Sinif
2    22147
3    16956
4    13113
1    10260
0     1489
5      961
6      177
7       22
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Sinif':
0
 
count    65125.000000
mean         2.516622
std          1.110235
min          0.000000
25%          2.000000
50%          2.000000
75%          3.000000
max          7.000000
Name: Sinif, dtype: float64


### 2.11-) Universite Not Ortalamasi

In [50]:
ortalama_mapping = {'ORTALAMA BULUNMUYOR': 0, 
                   'Not ortalaması yok': 0, 
                   'Ortalama bulunmuyor': 0,
                   'Hazırlığım': 0,
                   '0 - 1.79': 0, 
                   '1.00 - 2.50':1, 
                   '1.80 - 2.49':1, 
                   '2.50 ve altı':1, 
                   '2.00 - 2.50':1, 
                   "3.00-2.50":2,
                   "2.50 - 3.00":2, 
                   "2.50 - 2.99":2, 
                   '2.50 -3.00':2, 
                   '3.00 - 3.50':3, 
                   '3.50-3':3, 
                   '3.00 - 3.49':3, 
                   '3.00 - 4.00':3, 
                   '3.50 - 4.00':4, 
                   '4-3.5':4,
                   '4.0-3.5':4}


train_df = train_df.rename(columns={'Universite Not Ortalamasi': 'Not Ortalamasi'})
train_df['Not Ortalamasi'] = train_df['Not Ortalamasi'].replace(ortalama_mapping)
train_df['Not Ortalamasi'] = train_df['Not Ortalamasi'].fillna(2)

/var/folders/q7/w_hh_4ds07n_pl9zc0p1wxbc0000gn/T/ipykernel_8662/327464256.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['Not Ortalamasi'] = train_df['Not Ortalamasi'].replace(ortalama_mapping)


In [51]:
analyze_column(train_df, 'Not Ortalamasi')

Unique values in 'Not Ortalamasi':
[3. 2. 1. 4. 0.]
 
Value counts in 'Not Ortalamasi' (including NaNs):
Not Ortalamasi
2.0    23479
3.0    17885
1.0    14566
4.0     4944
0.0     4251
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Not Ortalamasi':
0
 
count    65125.000000
mean         2.072246
std          1.028516
min          0.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          4.000000
Name: Not Ortalamasi, dtype: float64


### 2.12-) Mezuniyet Durumu

In [52]:
train_df = train_df.rename(columns={'Daha Once Baska Bir Universiteden Mezun Olmus': 'Mezuniyet Durumu'})
train_df['Mezuniyet Durumu'] = train_df['Mezuniyet Durumu'].fillna('Hayır')
train_df['Mezuniyet Durumu'] = label_encoder.fit_transform(train_df['Mezuniyet Durumu'])
analyze_column(train_df, 'Mezuniyet Durumu')

Unique values in 'Mezuniyet Durumu':
[1 0]
 
Value counts in 'Mezuniyet Durumu' (including NaNs):
Mezuniyet Durumu
1    63969
0     1156
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Mezuniyet Durumu':
0
 
count    65125.000000
mean         0.982250
std          0.132044
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: Mezuniyet Durumu, dtype: float64


### 2.13-) Lise Adi

In [53]:
'''
92% of the column is missing -> drop
'''
analyze_column(train_df, 'Lise Adi')

Unique values in 'Lise Adi':
['Ankara Ataturk Anadolu Lisesi' 'Betül Can Anadolu Lisesi' 'Batem' ...
 'BAĞCILAR EDİP İPLİK MESLEKİ VE TEKNİK ANADOLU LİSESİ'
 'Ahmet Vefik paşa Anadolu lisesi'
 'Samsun Terme Karadeniz mesleki teknik Anadolu lisesi']
 
Value counts in 'Lise Adi' (including NaNs):
Lise Adi
------                                                                             4251
NaN                                                                                 384
Atatürk Lisesi                                                                      199
Açık öğretim lisesi                                                                 161
Açık Öğretim Lisesi                                                                 151
                                                                                   ... 
Nişantaşı Işık Lisesi                                                                 1
Tarsus atatürk lisesi                                                          

### 14 -) Lise Adi Diger

In [54]:
'''
92% of the column is missing -> drop
'''
analyze_column(train_df, 'Lise Adi Diger')

Unique values in 'Lise Adi Diger':
[nan 'İstanbul Alman Lisesi' 'şehzade mehmet koleji' ...
 'Gençosman İmkb tml' 'nefise andicen lisesi' 'büyük çiğli anadolu lisesi']
 
Value counts in 'Lise Adi Diger' (including NaNs):
Lise Adi Diger
NaN                                60392
Atatürk Anadolu Lisesi                 8
Beşiktaş Anadolu Lisesi                7
Açıköğretim Lisesi                     6
Uşak Fen Lisesi                        6
                                   ...  
OSMANGAZİ TİCARET MESLEK LİSESİ        1
Seyyid Burhaneddin ATL                 1
Süleyman Çelebi                        1
Bitls Ticaret meslek lisesi            1
büyük çiğli anadolu lisesi             1
Name: count, Length: 4011, dtype: int64
 
Number of missing values (NaNs) in 'Lise Adi Diger':
60392
 
count                       4733
unique                      4010
top       Atatürk Anadolu Lisesi
freq                           8
Name: Lise Adi Diger, dtype: object


### 2.16-) Lise Sehir

In [55]:
# Replace 'None' with 'İstanbul'
train_df['Lise Sehir'] = train_df['Lise Sehir'].apply(fill_city)
train_df['Lise Sehir'] = train_df['Lise Sehir'].fillna('İstanbul')
train_df['Lise Sehir'] = label_encoder.fit_transform(train_df['Lise Sehir'])
analyze_column(train_df, 'Lise Sehir')

Unique values in 'Lise Sehir':
[ 5 22 77 64 41 78 67 39 44 48 28 52  0 60 58  6 27 49 18  2 74 57 12  3
 61 16 20 11 50  9 43 37 15 45 46 65 76 73 66 29 23 32 79 21 51 36 53 47
 34 38 26 13 33 55 56  8 59 72  4 25 70 69 71 40 75 14 42 24 30 31 80 62
  1 19 63 10 17 68  7 54 35]
 
Value counts in 'Lise Sehir' (including NaNs):
Lise Sehir
77    19837
5      5165
20     2360
78     2248
0      1917
      ...  
14       68
75       55
7        50
68       47
31       43
Name: count, Length: 81, dtype: int64
 
Number of missing values (NaNs) in 'Lise Sehir':
0
 
count    65125.000000
mean        48.127616
std         28.083151
min          0.000000
25%         21.000000
50%         52.000000
75%         77.000000
max         80.000000
Name: Lise Sehir, dtype: float64


### 2.17-) Lise Turu

In [56]:
lise_turu_mapping = {'İmam Hatip Lisesi': 0, 
                     'Diğer': 0, 
                     'Devlet': 0,
                     'Düz lise': 0, 
                     'Düz Lise': 0, 
                     'Meslek lisesi':1, 
                     'Meslek':1, 
                     'Meslek Lisesi':1, 
                     'Özel':1, 
                     'Özel Lisesi':1, 
                     'Özel lisesi':1, 
                     'Özel Lise':1,
                     'Anadolu Lisesi':2,
                     'Anadolu lisesi':2,
                     'Fen lisesi':3,
                     'Fen Lisesi':3}

train_df['Lise Turu'] = train_df['Lise Turu'].replace(lise_turu_mapping)
train_df['Lise Turu'] = train_df['Lise Turu'].fillna(0)
analyze_column(train_df, 'Lise Turu')

Unique values in 'Lise Turu':
[2. 1. 3. 0.]
 
Value counts in 'Lise Turu' (including NaNs):
Lise Turu
0.0    28361
2.0    20369
1.0    13576
3.0     2819
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Lise Turu':
0
 
count    65125.000000
mean         0.963854
std          0.959221
min          0.000000
25%          0.000000
50%          1.000000
75%          2.000000
max          3.000000
Name: Lise Turu, dtype: float64


/var/folders/q7/w_hh_4ds07n_pl9zc0p1wxbc0000gn/T/ipykernel_8662/323145159.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['Lise Turu'] = train_df['Lise Turu'].replace(lise_turu_mapping)


### 2.17-) Lise Bolumu

In [57]:
most_common_lise_bolumu = train_df['Lise Bolumu'].value_counts().head(75).index
most_common_lise_bolumu

Index(['Sayısal', 'Eşit Ağırlık', 'FEN SAYISAL BİLİMLERİ ALANI', 'Sözel',
       'EŞİT AĞIRLIK TÜRKÇE-MATEMATİK ALANI', 'Dil', 'Diğer', 'Yabancı Dil',
       'TM', 'SOSYAL ALANI', 'sayısal', 'Fen Bilimleri', 'MF', 'eşit ağırlık',
       'SAYISAL', 'tm', 'DİL ALANI', 'Türkçe-Matematik', 'Fen',
       'FEN BİLİMLERİ', 'Eşit ağırlık', 'EŞİT AĞIRLIK', 'Matematik-Fen',
       'fen bilimleri', 'Tm', 'fen', 'Bilişim Teknolojileri', 'sayisal',
       'Türkçe Matematik', 'TÜRKÇE-MATEMATİK', 'mf', 'Fen bilimleri',
       'türkçe-matematik', 'FEN', 'türkçe matematik', 'sözel', 'Sayisal', 'EA',
       'TÜRKÇE MATEMATİK', 'Fen-Matematik', 'Türkçe - Matematik', '-',
       'Türkçe matematik', 'yok', 'Sosyal Bilimler', 'Bilgisayar',
       'yabancı dil', 'Matematik - Fen', 'SOSYAL BİLİMLER', 'sosyal bilimler',
       'Mf', 'SÖZEL', 'Türkçe-matematik', 'esit agirlik', 'Esit agirlik',
       'bilişim teknolojileri', 'FM', 'Elektrik-Elektronik', 'matematik fen',
       'BİLİŞİM TEKNOLOJİLERİ', 'MATEMATİ

In [58]:
simplified_mapping = {
    'Sayısal': [
        'Sayısal', 'SAYISAL', 'sayisal', 'MF', 'matematik fen', 'Fen-Matematik',
        'Matematik-Fen', 'TÜRKÇE-MATEMATİK', 'Türkçe-Matematik', 'Türkçe - Matematik',
        'Türkçe matematik', 'MF', 'matematik-fen', 'MATEMATİK-FEN', 'Fen Matematik',
        'Eşit Ağırlık', 'eşit ağırlık', 'EŞİT AĞIRLIK', 'esit agirlik', 'Esit agirlik',
        'EŞİT AĞIRLIK TÜRKÇE-MATEMATİK ALANI', 'Türkçe Matematik','Fen Bilimleri', 'FEN BİLİMLERİ', 'Fen', 'fen bilimleri', 'FEN',
        'Fen Matematik', 'Fen-Matematik', 'Fen Bilimleri', 'Matematik-Fen'
    ],
    
    'Sözel': [
        'Sözel', 'Sosyal Bilimler', 'SOSYAL BİLİMLER', 'sosyal bilimler', 'Sosyal Alanı',
        'Sosyal Bilimler ve Edebiyat', 'Eşit Ağırlık', 'Eşit Ağırlık Türkçe-Matematik Alanı'
    ],
    'Teknik': [
        'Bilgisayar', 'Bilişim Teknolojileri', 'Bilgisayar Programcılığı', 'Elektrik-Elektronik',
        'Bilgisayarlı Muhasebe', 'Muhasebe', 'Dış Ticaret', 'Web Tasarım', 'Bilgisayar Programcılığı'
    ],
    'Diger': [
        'Dil', 'Yabancı Dil', 'Diğer', 'Türkçe-Matematik', 'yok', 'bilişim teknolojileri',
        'Dış Ticaret', 'yabancı dil'
    ]
}

# Reverse the mapping for easier lookup
reverse_mapping = {item: category for category, items in simplified_mapping.items() for item in items}

# Apply the mapping to the DataFrame without filling NaNs
train_df['Lise Bolumu'] = train_df['Lise Bolumu'].map(reverse_mapping)
train_df['Lise Bolumu'] = train_df['Lise Bolumu'].fillna('Sayısal')
train_df['Lise Bolumu'] = label_encoder.fit_transform(train_df['Lise Bolumu'])
analyze_column(train_df, 'Lise Bolumu')

Unique values in 'Lise Bolumu':
[1 2 3 0]
 
Value counts in 'Lise Bolumu' (including NaNs):
Lise Bolumu
1    42132
2    20621
0     2234
3      138
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Lise Bolumu':
0
 
count    65125.000000
mean         1.286572
std          0.526590
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          3.000000
Name: Lise Bolumu, dtype: float64


### 2.18-) Lise Bolum Diger

In [59]:
'''
90% of the column is missing -> drop
'''
analyze_column(train_df, 'Lise Bolum Diger')

Unique values in 'Lise Bolum Diger':
[nan '-' 'ÖZEL FEN LİSESİ'
 'Sosyal Bilimleri Lisesi seçeneği koymadığınız için Fen Lisesi sekmesini işaretledim.'
 'Grafik Ve Fotoğraf' 'Bilişim Teknolojileri' 'mezuniyet notu:80'
 'Çocuk gelişimi ve eğitimi' 'yabancı dili:almanca' 'Ticaret'
 'devlet lisesi' 'Bilgisayarlı Muhasebe bölümü' 'Güverte Yönetimi'
 'Gazetecilik' 'Bakalorya programı' 'Eşit ağırlık devlet lisesi'
 'Bilgisayarlı Muhasebe' 'Alman Bölümü' 'Muhasebe ve Finansman'
 'ANADOLU ÖĞRETMEN LİSESİ' 'turizm ve otelcilik' 'dil bölümü'
 'halkla ilişkiler' 'IB High level : Fizik Matematik' 'Pazarlama'
 'MUHASEBE' 'İnşaat Teknolojisi' 'ACİL TIP TEKNİKERLİĞİ'
 'Grafik Tasarım ve Fotoğrafçılık' 'elektrik ve elektronik'
 'görüntü ve ses' 'Uluslararası Bakalorya Diploma Programı'
 'ÇOCUK GELİŞİMİ'
 'İlk üç sene sayısal (1 yıl International Baccalaureate), son yıl regular programda eşit ağırlık'
 'Genel' 'Bilgisayar Programcılığı'
 '3sene sayısal , son sene eşit ağırlık' 'Grafik ve Fotoğraf'
 'Bi

### 2.19-) Lise Mezuniyet Notu

In [60]:
analyze_column(train_df, 'Lise Mezuniyet Notu')

Unique values in 'Lise Mezuniyet Notu':
['3.50-3' '3.00 - 4.00' '3.00-2.50' '2.50 ve altı' '4.00-3.50' '3.50-3.00'
 nan '69-55' '100-85' '84-70' '54-45' '44-0' '75 - 100' '50 - 75'
 'Not ortalaması yok' '25 - 50' '0 - 25' '50 - 74' '25 - 49' '0 - 24']
 
Value counts in 'Lise Mezuniyet Notu' (including NaNs):
Lise Mezuniyet Notu
75 - 100              22433
84-70                  9141
100-85                 6967
NaN                    5917
4.00-3.50              3689
50 - 75                3115
3.00 - 4.00            3101
3.50-3.00              2759
3.50-3                 2330
69-55                  2162
3.00-2.50              2153
50 - 74                1007
2.50 ve altı            226
54-45                    88
25 - 50                  14
Not ortalaması yok       11
44-0                      6
0 - 25                    3
25 - 49                   2
0 - 24                    1
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Lise Mezuniyet Notu':
5917
 
count        5920

In [61]:
lise_not_mapping = {
    # Highest Scores
    '75 - 100': 5,
    '100-85': 5,
    '4.00-3.50': 5,
    
    # High Scores
    '84-70': 4,
    '3.00 - 4.00': 4,
    '3.50-3.00': 4,
    '3.50-3': 4,
    
    # Mid Scores
    '50 - 75': 3,
    '69-55': 3,
    '3.00-2.50': 3,
    '50 - 74': 3,
    
    # Low Scores
    '2.50 ve altı': 2,
    '54-45': 2,
    '25 - 50': 1,
    
    # No Scores
    'Not ortalaması yok': 0,
    '44-0': 0,
    '0 - 25': 0,
    '25 - 49': 1,
    '0 - 24': 0
}

train_df['Lise Mezuniyet Notu'] = train_df['Lise Mezuniyet Notu'].map(lise_not_mapping)
train_df['Lise Mezuniyet Notu'] = train_df['Lise Mezuniyet Notu'].fillna(5)
analyze_column(train_df, 'Lise Mezuniyet Notu')

Unique values in 'Lise Mezuniyet Notu':
[4. 3. 2. 5. 0. 1.]
 
Value counts in 'Lise Mezuniyet Notu' (including NaNs):
Lise Mezuniyet Notu
5.0    39006
4.0    17331
3.0     8437
2.0      314
0.0       21
1.0       16
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Lise Mezuniyet Notu':
0
 
count    65125.00000
mean         4.45772
std          0.73868
min          0.00000
25%          4.00000
50%          5.00000
75%          5.00000
max          5.00000
Name: Lise Mezuniyet Notu, dtype: float64


### 2-20-) Baska Bir Kurumdan Burs Aliyor mu?

In [62]:
train_df['Baska Bir Kurumdan Burs Aliyor mu?'] = train_df['Baska Bir Kurumdan Burs Aliyor mu?'].fillna('Hayır')
train_df['Baska Bir Kurumdan Burs Aliyor mu?'] = label_encoder.fit_transform(train_df['Baska Bir Kurumdan Burs Aliyor mu?'])
analyze_column(train_df, 'Baska Bir Kurumdan Burs Aliyor mu?')

Unique values in 'Baska Bir Kurumdan Burs Aliyor mu?':
[1 0]
 
Value counts in 'Baska Bir Kurumdan Burs Aliyor mu?' (including NaNs):
Baska Bir Kurumdan Burs Aliyor mu?
1    48744
0    16381
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Baska Bir Kurumdan Burs Aliyor mu?':
0
 
count    65125.000000
mean         0.748468
std          0.433897
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: Baska Bir Kurumdan Burs Aliyor mu?, dtype: float64


### 2.21-) Burs Aldigi Baska Kurum

In [63]:
train_df['Burs Aldigi Baska Kurum'] = train_df['Burs Aldigi Baska Kurum'].fillna('-')
train_df['Burs Aldigi Baska Kurum'] = train_df['Burs Aldigi Baska Kurum'].str.lower()
train_df['Burs Aldigi Baska Kurum'] = train_df['Burs Aldigi Baska Kurum'].apply(lambda x: 0 if x == '-' else (1 if 'kyk' in x else 2))
analyze_column(train_df, 'Burs Aldigi Baska Kurum')

Unique values in 'Burs Aldigi Baska Kurum':
[0 1 2]
 
Value counts in 'Burs Aldigi Baska Kurum' (including NaNs):
Burs Aldigi Baska Kurum
0    48688
1     8757
2     7680
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Burs Aldigi Baska Kurum':
0
 
count    65125.000000
mean         0.370319
std          0.684868
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          2.000000
Name: Burs Aldigi Baska Kurum, dtype: float64


### 2.22-) Burs Miktari

In [64]:
train_df = train_df.rename(columns={'Baska Kurumdan Aldigi Burs Miktari': 'Burs Miktari'})
train_df['Burs Miktari'] = train_df['Burs Miktari'].apply(extract_numbers)
analyze_column(train_df, 'Burs Miktari')

Unique values in 'Burs Miktari':
[    0   300   150   430   400   250  1000   850   500   280   900   365
   125   180   260   425   100   800   200   325   275   600   475   675
   750   330   490   320   560   350   720   590  1990  1160   535   700
  1500  2000   620   175   570  1200  1750   610   520   460   515  1100
   820   815   450   240   470    60   670   315   480   650   780   920
   440  1275   340   660   130  1120   420   375   514   225  1075   220
  2017   424   435   640  3000   120   485   550  1080   255  2700   665
   290  1250  1300 14500  2200   970  2300   525  1600    75   423    80
   410   270   170   140   580   390   124    50   415  2500   385    25
     5    72    70  5000   295   360   380     9   160    40   135     4
   840   210   370   990    90   185   137   265   530   115   310  2015
   775   110   950    35 20010  3675   165  1550  2016   725   426   433
   245   221    97   540  4225   215   230  1800   355    95  4200  2012
  1450   790  1410

### 2.23-) Anne Egitim Durumu

In [65]:
education_mapping = {
    "i̇lkokul mezunu": 1,
    "i̇lkokul": 1,
    "lise": 3,
    "li̇se": 3,
    "lise mezunu": 3,
    "üniversite": 4,
    "üni̇versi̇te": 4,
    "üniversite mezunu": 4,
    "ortaokul mezunu": 2,
    "ortaokul": 2,
    "eğitimi yok": 0,
    "eği̇ti̇m yok": 0,
    "eğitim yok": 0,
    "yüksek lisans": 5,
    "yüksek li̇sans": 5,
    "doktora": 5,
    "yüksek lisans / doktora": 5,
    "yüksek lisans / doktara": 5
}

train_df['Anne Egitim Durumu'] = train_df['Anne Egitim Durumu'].str.lower()
train_df['Anne Egitim Durumu'] = train_df['Anne Egitim Durumu'].map(education_mapping)
train_df['Anne Egitim Durumu'] = train_df['Anne Egitim Durumu'].fillna(1)
analyze_column(train_df, 'Anne Egitim Durumu')

Unique values in 'Anne Egitim Durumu':
[4. 1. 0. 3. 2. 5.]
 
Value counts in 'Anne Egitim Durumu' (including NaNs):
Anne Egitim Durumu
1.0    26557
3.0    13395
4.0     9012
0.0     8351
2.0     7033
5.0      777
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Anne Egitim Durumu':
0
 
count    65125.000000
mean         1.853988
std          1.328907
min          0.000000
25%          1.000000
50%          1.000000
75%          3.000000
max          5.000000
Name: Anne Egitim Durumu, dtype: float64


### 2.24-) Anne Calisma Durumu

In [66]:
train_df['Anne Calisma Durumu'] = train_df['Anne Calisma Durumu'].fillna('Hayır')
train_df['Anne Calisma Durumu'] = label_encoder.fit_transform(train_df['Anne Calisma Durumu'])
analyze_column(train_df, 'Anne Calisma Durumu')

Unique values in 'Anne Calisma Durumu':
[2 1 0]
 
Value counts in 'Anne Calisma Durumu' (including NaNs):
Anne Calisma Durumu
2    51267
1    10504
0     3354
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Anne Calisma Durumu':
0
 
count    65125.000000
mean         1.735708
std          0.545388
min          0.000000
25%          2.000000
50%          2.000000
75%          2.000000
max          2.000000
Name: Anne Calisma Durumu, dtype: float64


### 2.24-) Anne Sektor

In [67]:
anne_sektor_mapping = {
    '0': 0,
    '-': 0,
    'diger': 1,
    'diğer': 1,
    'kamu': 2,
    'özel sektör': 3
}

train_df['Anne Sektor'] = train_df['Anne Sektor'].str.lower()
train_df['Anne Sektor'] = train_df['Anne Sektor'].map(anne_sektor_mapping)
train_df['Anne Sektor'] = train_df['Anne Sektor'].fillna(0)
analyze_column(train_df, 'Anne Sektor')

Unique values in 'Anne Sektor':
[0. 2. 3. 1.]
 
Value counts in 'Anne Sektor' (including NaNs):
Anne Sektor
0.0    52601
3.0     6042
2.0     5149
1.0     1333
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Anne Sektor':
0
 
count    65125.000000
mean         0.456921
std          0.981294
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          3.000000
Name: Anne Sektor, dtype: float64


### 2.25-) Baba Egitim Durumu

In [68]:
baba_education_mapping = {
    "eğitimi yok": 0,
    "eği̇ti̇m yok": 0,
    "eğitim yok": 0,
    "0": 0,  # for the value 0
    
    "i̇lkokul mezunu": 1,
    "i̇lkokul": 1,
    
    "ortaokul mezunu": 2,
    "ortaokul": 2,
    
    "lise": 3,
    "li̇se": 3,
    "lise mezunu": 3,
    
    "üniversite": 4,
    "üni̇versi̇te": 4,
    "üniversite mezunu": 4,
    
    "yüksek lisans": 5,
    "yüksek li̇sans": 5,
    "doktora": 5,
    "yüksek lisans / doktora": 5,
    "yüksek lisans / doktara": 5
}

train_df['Baba Egitim Durumu'] = train_df['Baba Egitim Durumu'].str.lower()
train_df['Baba Egitim Durumu'] = train_df['Baba Egitim Durumu'].map(baba_education_mapping)
train_df['Baba Egitim Durumu'] = train_df['Baba Egitim Durumu'].fillna(1)
analyze_column(train_df, 'Baba Egitim Durumu')

Unique values in 'Baba Egitim Durumu':
[4. 5. 1. 2. 3. 0.]
 
Value counts in 'Baba Egitim Durumu' (including NaNs):
Baba Egitim Durumu
1.0    20437
3.0    15895
4.0    15138
2.0     9180
0.0     2609
5.0     1866
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Baba Egitim Durumu':
0
 
count    65125.000000
mean         2.400983
std          1.320992
min          0.000000
25%          1.000000
50%          3.000000
75%          4.000000
max          5.000000
Name: Baba Egitim Durumu, dtype: float64


### 2.26-) Baba Calisma Durumu

In [69]:
train_df['Baba Calisma Durumu'] = train_df['Baba Calisma Durumu'].fillna('Hayır')
train_df['Baba Calisma Durumu'] = label_encoder.fit_transform(train_df['Baba Calisma Durumu'])
analyze_column(train_df, 'Baba Calisma Durumu')

Unique values in 'Baba Calisma Durumu':
[0 1 2]
 
Value counts in 'Baba Calisma Durumu' (including NaNs):
Baba Calisma Durumu
1    28559
2    25104
0    11462
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Baba Calisma Durumu':
0
 
count    65125.000000
mean         1.209474
std          0.719446
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          2.000000
Name: Baba Calisma Durumu, dtype: float64


### 2.27-) Baba Sektor

In [70]:
baba_sektor_mapping = {
    '0': 0,
    '-': 0,
    'diger': 1,
    'diğer': 1,
    'kamu': 2,
    'özel sektör': 3
}

train_df['Baba Sektor'] = train_df['Baba Sektor'].str.lower()
train_df['Baba Sektor'] = train_df['Baba Sektor'].map(baba_sektor_mapping)
train_df['Baba Sektor'] = train_df['Baba Sektor'].fillna(0)
analyze_column(train_df, 'Baba Sektor')

Unique values in 'Baba Sektor':
[0. 3. 2. 1.]
 
Value counts in 'Baba Sektor' (including NaNs):
Baba Sektor
0.0    32476
3.0    15613
2.0    10319
1.0     6717
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Baba Sektor':
0
 
count    65125.000000
mean         1.139255
std          1.263610
min          0.000000
25%          0.000000
50%          1.000000
75%          2.000000
max          3.000000
Name: Baba Sektor, dtype: float64


### 2.28-) Kardes Sayisi

In [71]:
kardes_mapping = {
    0: 0,
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    6: 6,
    7: 7,
    8: 8,
    9: 9,
    10: 10,
    11: 11,
    12: 12,
    13: 13,
    14: 14,
    0.0: 0,
    1.0: 1,
    2.0: 2,
    3.0: 3,
    4.0: 4,
    5.0: 5,
    6.0: 6,
    7.0: 7,
    8.0: 8,
    9.0: 9,
    10.0: 10,
    11.0: 11,
    12.0: 12,
    13.0: 13,
    14.0: 14,
    18: 18,
    'Kardeş Sayısı 1 Ek Bilgi Aile Hk. Anne Vefat': 19,
}

train_df['Kardes Sayisi'] = train_df['Kardes Sayisi'].map(kardes_mapping)
train_df['Kardes Sayisi'] = train_df['Kardes Sayisi'].fillna(1) # mod
analyze_column(train_df, 'Kardes Sayisi')

Unique values in 'Kardes Sayisi':
[ 1. 19.  4.  5.  2.  6.  3.  9.  0. 10.  7.  8.]
 
Value counts in 'Kardes Sayisi' (including NaNs):
Kardes Sayisi
1.0     32926
2.0     13062
3.0      6734
4.0      4611
0.0      3701
5.0      3421
6.0       206
7.0       159
10.0      127
8.0        98
9.0        79
19.0        1
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Kardes Sayisi':
0
 
count    65125.000000
mean         1.841597
std          1.386260
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         19.000000
Name: Kardes Sayisi, dtype: float64


### 2.28-) Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?

In [72]:
train_df['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = train_df['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].fillna('Hayır')
train_df['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = label_encoder.fit_transform(train_df['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'])
analyze_column(train_df, 'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?')

Unique values in 'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?':
[0 1]
 
Value counts in 'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?' (including NaNs):
Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?
1    35539
0    29586
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?':
0
 
count    65125.000000
mean         0.545704
std          0.497911
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?, dtype: float64


### 2.29-) Uye Oldugunuz Kulubun Ismi

In [73]:
train_df['Uye Oldugunuz Kulubun Ismi'] = label_encoder.fit_transform(train_df['Uye Oldugunuz Kulubun Ismi'])
analyze_column(train_df, 'Uye Oldugunuz Kulubun Ismi') # YA DA DROP BELLI DEGIL 48 K BOS 

Unique values in 'Uye Oldugunuz Kulubun Ismi':
[ 1401 13423  8771 ...  7725  5786  4000]
 
Value counts in 'Uye Oldugunuz Kulubun Ismi' (including NaNs):
Uye Oldugunuz Kulubun Ismi
13423    48691
4818       203
4248        60
5542        43
212         40
         ...  
5344         1
5960         1
13189        1
12313        1
4000         1
Name: count, Length: 13424, dtype: int64
 
Number of missing values (NaNs) in 'Uye Oldugunuz Kulubun Ismi':
0
 
count    65125.000000
mean     11726.717482
std       3501.371706
min          0.000000
25%      13323.000000
50%      13423.000000
75%      13423.000000
max      13423.000000
Name: Uye Oldugunuz Kulubun Ismi, dtype: float64


### 2.30-) Profesyonel Bir Spor Daliyla Mesgul musunuz?

In [74]:
train_df['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = train_df['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].fillna('Hayır')
train_df['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = label_encoder.fit_transform(train_df['Profesyonel Bir Spor Daliyla Mesgul musunuz?'])
analyze_column(train_df, 'Profesyonel Bir Spor Daliyla Mesgul musunuz?')

Unique values in 'Profesyonel Bir Spor Daliyla Mesgul musunuz?':
[0 1]
 
Value counts in 'Profesyonel Bir Spor Daliyla Mesgul musunuz?' (including NaNs):
Profesyonel Bir Spor Daliyla Mesgul musunuz?
1    43358
0    21767
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Profesyonel Bir Spor Daliyla Mesgul musunuz?':
0
 
count    65125.000000
mean         0.665766
std          0.471726
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: Profesyonel Bir Spor Daliyla Mesgul musunuz?, dtype: float64


### 2.31-) Spor Dalindaki Rolunuz Nedir?

In [75]:
rol_mapping = {
    '0': 0,
    '-': 0,
    'Diger': 0,
    'DİĞER': 0,
    'Bireysel': 1,
    'Takım Oyuncusu': 2,
    'KAPTAN / LİDER': 3,
    'Lider/Kaptan': 3,
    'Kaptan': 3
}

train_df['Spor Dalindaki Rolunuz Nedir?'] = train_df['Spor Dalindaki Rolunuz Nedir?'].map(rol_mapping)
train_df['Spor Dalindaki Rolunuz Nedir?'] = train_df['Spor Dalindaki Rolunuz Nedir?'].fillna(0)
analyze_column(train_df, 'Spor Dalindaki Rolunuz Nedir?')

Unique values in 'Spor Dalindaki Rolunuz Nedir?':
[0. 3. 1. 2.]
 
Value counts in 'Spor Dalindaki Rolunuz Nedir?' (including NaNs):
Spor Dalindaki Rolunuz Nedir?
0.0    58544
3.0     3709
1.0     1697
2.0     1175
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Spor Dalindaki Rolunuz Nedir?':
0
 
count    65125.000000
mean         0.232998
std          0.745999
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          3.000000
Name: Spor Dalindaki Rolunuz Nedir?, dtype: float64


### 2.32-) Aktif olarak bir STK üyesi misiniz?

In [76]:
train_df['Aktif olarak bir STK üyesi misiniz?'] = train_df['Aktif olarak bir STK üyesi misiniz?'].fillna('Hayır')
train_df['Aktif olarak bir STK üyesi misiniz?'] = label_encoder.fit_transform(train_df['Aktif olarak bir STK üyesi misiniz?'])
analyze_column(train_df, 'Aktif olarak bir STK üyesi misiniz?')

Unique values in 'Aktif olarak bir STK üyesi misiniz?':
[1 0]
 
Value counts in 'Aktif olarak bir STK üyesi misiniz?' (including NaNs):
Aktif olarak bir STK üyesi misiniz?
1    50558
0    14567
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Aktif olarak bir STK üyesi misiniz?':
0
 
count    65125.000000
mean         0.776322
std          0.416712
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: Aktif olarak bir STK üyesi misiniz?, dtype: float64


### 2.33-) Hangi STK'nin Uyesisiniz?

In [77]:
stk_mapping = {
    'Kızılay': 'Kızılay',
    'KIZILAY': 'Kızılay',
    'kızılay': 'Kızılay',
    'Türk Kızılayı': 'Kızılay',
    'KIZILAY': 'Kızılay',
    'KIZILAY': 'Kızılay',
    '-': 'Unknown',
    'Tema': 'TEMA',
    'TEMA': 'TEMA',
    'TEMA Vakfı': 'TEMA',
    'Tema Vakfı': 'TEMA',
    'TEMA VAKFI': 'TEMA',
    'tema': 'TEMA',
    'tema vakfı': 'TEMA',
    'Tegv': 'TEGV',
    'TEGV': 'TEGV',
    'toplum gönüllüleri': 'Toplum Gönüllüleri',
    'Toplum Gönüllüleri': 'Toplum Gönüllüleri',
    'Toplum Gönüllüleri Vakfı': 'Toplum Gönüllüleri',
    'Toplum gönüllüleri vakfı': 'Toplum Gönüllüleri',
    'Toplum Gönüllüleri Vakfı': 'Toplum Gönüllüleri',
    'toplum gönüllüleri vakfı': 'Toplum Gönüllüleri',
    'TOG': 'TOG',
    'Tog': 'TOG',
    'AIESEC': 'AIESEC',
    'Aiesec': 'AIESEC',
    'Yeşilay': 'Yeşilay',
    'yeşilay': 'Yeşilay',
    'AFAD': 'AFAD',
    'Afad': 'AFAD',
    'Sivil Yaşam Derneği': 'Sivil Yaşam Derneği',
    'Çağdaş Yaşamı Destekleme Derneği': 'Çağdaş Yaşamı Destekleme Derneği',
    'Çağdaş yaşamı destekleme derneği': 'Çağdaş Yaşamı Destekleme Derneği',
    'Ahbap': 'Ahbap',
    'Greenpeace': 'Greenpeace',
    'YGA': 'YGA',
    'Young Guru Academy': 'YGA',
    'Türkiye Eğitim Gönüllüleri Vakfı': 'Türkiye Eğitim Gönüllüleri Vakfı',
    'İHH': 'İHH',
    'Habitat Derneği': 'Habitat Derneği',
    'Habitat derneği': 'Habitat Derneği',
    'Habitat': 'Habitat Derneği',
    'Anadolu Gençlik Derneği': 'Anadolu Gençlik Derneği',
    'IEEE': 'IEEE'
}


train_df['Hangi STK\'nin Uyesisiniz?'] = train_df['Hangi STK\'nin Uyesisiniz?'].map(stk_mapping).fillna('Other')
analyze_column(train_df, 'Hangi STK\'nin Uyesisiniz?')

In [78]:
train_df['Hangi STK\'nin Uyesisiniz?'] = label_encoder.fit_transform(train_df['Hangi STK\'nin Uyesisiniz?'])
analyze_column(train_df, 'Hangi STK\'nin Uyesisiniz?') # Standar scale or drop ?

Unique values in 'Hangi STK'nin Uyesisiniz?':
[9490 1583 8435 ... 1743 2490 2303]
 
Value counts in 'Hangi STK'nin Uyesisiniz?' (including NaNs):
Hangi STK'nin Uyesisiniz?
9490    49518
10       2956
3770      238
6623      196
5859      181
        ...  
1670        1
9412        1
4825        1
2365        1
2303        1
Name: count, Length: 9491, dtype: int64
 
Number of missing values (NaNs) in 'Hangi STK'nin Uyesisiniz?':
0
 
count    65125.000000
mean      8162.293390
std       2798.007367
min          0.000000
25%       9490.000000
50%       9490.000000
75%       9490.000000
max       9490.000000
Name: Hangi STK'nin Uyesisiniz?, dtype: float64


### 2.34-) Stk Projesine Katildiniz Mi?

In [79]:
train_df['Stk Projesine Katildiniz Mi?'] = train_df['Stk Projesine Katildiniz Mi?'].fillna('Hayır')
train_df['Stk Projesine Katildiniz Mi?'] = label_encoder.fit_transform(train_df['Stk Projesine Katildiniz Mi?'])
analyze_column(train_df, 'Stk Projesine Katildiniz Mi?')

Unique values in 'Stk Projesine Katildiniz Mi?':
[0 1]
 
Value counts in 'Stk Projesine Katildiniz Mi?' (including NaNs):
Stk Projesine Katildiniz Mi?
1    47969
0    17156
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Stk Projesine Katildiniz Mi?':
0
 
count    65125.000000
mean         0.736568
std          0.440498
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: Stk Projesine Katildiniz Mi?, dtype: float64


### 2.35-) Girisimcilikle Ilgili Deneyiminiz Var Mi?

In [80]:
train_df['Girisimcilikle Ilgili Deneyiminiz Var Mi?'] = train_df['Girisimcilikle Ilgili Deneyiminiz Var Mi?'].fillna('Hayır')
train_df['Girisimcilikle Ilgili Deneyiminiz Var Mi?'] = label_encoder.fit_transform(train_df['Girisimcilikle Ilgili Deneyiminiz Var Mi?'])
analyze_column(train_df, 'Girisimcilikle Ilgili Deneyiminiz Var Mi?')

Unique values in 'Girisimcilikle Ilgili Deneyiminiz Var Mi?':
[1 0]
 
Value counts in 'Girisimcilikle Ilgili Deneyiminiz Var Mi?' (including NaNs):
Girisimcilikle Ilgili Deneyiminiz Var Mi?
1    46759
0    18366
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Girisimcilikle Ilgili Deneyiminiz Var Mi?':
0
 
count    65125.000000
mean         0.717988
std          0.449982
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: Girisimcilikle Ilgili Deneyiminiz Var Mi?, dtype: float64


### 2.36-) Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?

In [81]:
train_df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'] = train_df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].fillna('-')
train_df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'] = train_df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].apply(lambda x: 0 if x == '-' else 1)
analyze_column(train_df, 'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?')

Unique values in 'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?':
[0 1]
 
Value counts in 'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?' (including NaNs):
Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?
0    55722
1     9403
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?':
0
 
count    65125.000000
mean         0.144384
std          0.351481
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?, dtype: float64


### 2.37-) Ingilizce Biliyor musunuz?

In [82]:
train_df['Ingilizce Biliyor musunuz?'] = train_df['Ingilizce Biliyor musunuz?'].fillna('Evet')
train_df['Ingilizce Biliyor musunuz?'] = label_encoder.fit_transform(train_df['Ingilizce Biliyor musunuz?'])
analyze_column(train_df, 'Ingilizce Biliyor musunuz?')

Unique values in 'Ingilizce Biliyor musunuz?':
[0 1]
 
Value counts in 'Ingilizce Biliyor musunuz?' (including NaNs):
Ingilizce Biliyor musunuz?
0    55326
1     9799
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Ingilizce Biliyor musunuz?':
0
 
count    65125.000000
mean         0.150464
std          0.357529
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: Ingilizce Biliyor musunuz?, dtype: float64


### 2.38-) Ingilizce Seviyeniz?

In [83]:
analyze_column(train_df, 'Ingilizce Seviyeniz?')

Unique values in 'Ingilizce Seviyeniz?':
[nan 'İleri' 'Orta' '0' 'Başlangıç']
 
Value counts in 'Ingilizce Seviyeniz?' (including NaNs):
Ingilizce Seviyeniz?
NaN          36533
Orta         13057
İleri         8318
Başlangıç     5180
0             2037
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Ingilizce Seviyeniz?':
36533
 
count     28592
unique        4
top        Orta
freq      13057
Name: Ingilizce Seviyeniz?, dtype: object


In [84]:
ing_mapping = {
    'Başlangıç': 1,
    'Orta': 2,
    'İleri': 3,
    '0': 0,
}

train_df['Ingilizce Seviyeniz?'] = train_df['Ingilizce Seviyeniz?'].map(ing_mapping)
train_df['Ingilizce Seviyeniz?'] = train_df['Ingilizce Seviyeniz?'].fillna(0)
analyze_column(train_df, 'Ingilizce Seviyeniz?')

Unique values in 'Ingilizce Seviyeniz?':
[0. 3. 2. 1.]
 
Value counts in 'Ingilizce Seviyeniz?' (including NaNs):
Ingilizce Seviyeniz?
0.0    38570
2.0    13057
3.0     8318
1.0     5180
Name: count, dtype: int64
 
Number of missing values (NaNs) in 'Ingilizce Seviyeniz?':
0
 
count    65125.000000
mean         0.863693
std          1.133610
min          0.000000
25%          0.000000
50%          0.000000
75%          2.000000
max          3.000000
Name: Ingilizce Seviyeniz?, dtype: float64


### 2.39-) Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite

In [86]:
analyze_column(train_df, 'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite')

Unique values in 'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite':
[nan 'Sabancı Üniversitesi' 'Anadolu Üniversitesi'
 'Mimar Sinan Güzel Sanatlar Üniversitesi' 'Adıyaman Üniversitesi'
 'TOBB Ekonomi ve Teknoloji Üniversitesi' 'Yalova Üniversitesi'
 'İSTANBUL ÜNİVERSİTESİ' 'ATATÜRK ÜNİVERSİTESİ' 'DOĞUŞ ÜNİVERSİTESİ'
 'MARMARA ÜNİVERSİTESİ' 'SAKARYA ÜNİVERSİTESİ' 'ÇANAKKALE ONSEKİZ MART'
 'KOCAELİ ÜNİVERSİTESİ' 'KARABÜK ÜNİVERSİTESİ' 'ANADOLU ÜNİVERSİTESİ'
 'ÇUKUROVA ÜNİVERSİTESİ' 'İZMİR EKONOMİ ÜNİVERSİTESİ'
 'TRAKYA ÜNİVERSİTESİ' 'İSTANBUL KÜLTÜR ÜNİVERSİTESİ'
 'ÜSKÜDAR ÜNİVERSİTESİ' 'EGE ÜNİVERSİTESİ' 'OKAN ÜNİVERSİTESİ'
 'BEYKOZ LOJİSTİK MESLEK' 'Düzce Üniversitesi' 'Marmara Üniversitesi'
 'Bahçeşehir Üniversitesi' 'AVRASYA ÜNİVERSİTESİ' 'PİRİ REİS ÜNİVERSİTESİ'
 'İSTANBUL MEDİPOL ÜNİVERSİTESİ' 'Çukurova Üniversitesi'
 'Bozok Üniversitesi' 'Siirt Üniversitesi' 'Yıldız Teknik Üniversitesi'
 'İstanbul Gelişim Üniversitesi' 'Yüzüncü Yıl Üniversitesi'
 'Okan Üniversitesi' 'MİMAR 

In [87]:
train_df['Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite'] = label_encoder.fit_transform(train_df['Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite'])
scaler = StandardScaler()
train_df['Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite'] = scaler.fit_transform(train_df[['Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite']])

In [88]:
analyze_dataframe(train_df)

SHAPE
(65125, 44)

----------------------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65125 entries, 0 to 65124
Data columns (total 44 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Basvuru Yili                                               65125 non-null  int64  
 1   Degerlendirme Puani                                        65125 non-null  float64
 2   Cinsiyet                                                   65125 non-null  int64  
 3   Dogum Tarihi                                               65125 non-null  float64
 4   Dogum Yeri                                                 65125 non-null  int64  
 5   Ikametgah Sehri                                            65125 non-null  int64  
 6   Universite Adi                                             65125 non-null  int64  
 7   Universite Turu      